# Edge Search

Notebook to search for extreme heat events that overlap years.
Fix to resolve problem TBD.

By Cascade Tuholske, 2019-10-19


### Preliminary Findings

- In the entire record, there are 97 events that start on Jan 1.
- In the entire record, there are 94 events that end of Dec 31.

Of these, it looks like 5 were from the same city and bridged two years


#### Depdencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Load File

In [15]:
file_url = "https://www.dropbox.com/s/nxoffwsd135yznw/All_data20191019.csv?dl=1"  # Note: Need ?dl=1 to make sure this file gets read correctly
df = pd.read_csv(file_url)
df.columns

Index(['Unnamed: 0', 'Event_ID', 'ID_HDC_G0', 'CTR_MN_NM', 'total_days',
       'duration', 'avg_temp', 'avg_intensity', 'tot_intensity', 'event_dates',
       'intensity', 'tmax', 'year'],
      dtype='object')

In [16]:
df[df['ID_HDC_G0'] == 5534]

,Unnamed: 0,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year
22525,22525,22526,5534,Ethiopia,84,1,40.736885,0.136885,0.136885,['1983.01.05'],[0.136885],[40.736885],1983
22526,22526,22527,5534,Ethiopia,84,13,41.737855,1.137855,14.792109,['1983.01.19' '1983.01.20' '1983.01.21' '1983....,[0.846815 2.002577 1.19209 0.916884 0.81514 ...,[41.446815 42.602577 41.79209 41.516884 41.41...,1983
22527,22527,22528,5534,Ethiopia,84,9,41.062670,0.462670,4.164027,['1983.02.05' '1983.02.06' '1983.02.07' '1983....,[0.137976 0.035193 0.511267 0.594824 1.123145 ...,[40.737976 40.635193 41.111267 41.194824 41.72...,1983
22528,22528,22529,5534,Ethiopia,84,8,41.553627,0.953627,7.629014,['1983.02.16' '1983.02.17' '1983.02.18' '1983....,[0.600165 1.43894 1.109015 1.421606 1.34052 ...,[41.200165 42.03894 41.709015 42.021606 41.94...,1983
22529,22529,22530,5534,Ethiopia,84,43,42.115429,1.515429,65.163434,['1983.02.26' '1983.02.27' '1983.02.28' '1983....,[0.33683 1.354803 1.337897 1.908553 0.734724 ...,[40.93683 41.954803 41.937897 42.508553 41.33...,1983
22530,22530,22531,5534,Ethiopia,84,10,41.332326,0.732326,7.323264,['1983.04.11' '1983.04.12' '1983.04.13' '1983....,[0.224825 0.785494 0.962283 1.486514 0.874148 ...,[40.824825 41.385494 41.562283 42.086514 41.47...,1983
45818,23121,45819,5534,Ethiopia,103,2,41.258831,0.658831,1.317663,['1984.01.01' '1984.01.02'],[0.795733 0.52193 ],[41.395733 41.12193 ],1984
45819,23122,45820,5534,Ethiopia,103,3,41.405620,0.805620,2.416860,['1984.01.07' '1984.01.08' '1984.01.09'],[0.42183 1.36269 0.63234],[41.02183 41.96269 41.23234],1984
45820,23123,45821,5534,Ethiopia,103,2,41.278409,0.678409,1.356818,['1984.01.11' '1984.01.12'],[1.01482 0.341998],[41.61482 40.941998],1984
45821,23124,45822,5534,Ethiopia,103,20,41.422909,0.822909,16.458184,['1984.01.14' '1984.01.15' '1984.01.16' '1984....,[0.115893 0.989214 0.969042 0.85222 1.455157 ...,[40.715893 41.589214 41.569042 41.45222 42.05...,1984


#### Find Edges

Build query to find dates XXXX.12.31 and XXXX.01.01 
Events col are strings

In [21]:
df.dtypes

Unnamed: 0         int64
Event_ID           int64
ID_HDC_G0          int64
CTR_MN_NM         object
total_days         int64
duration           int64
avg_temp         float64
avg_intensity    float64
tot_intensity    float64
event_dates       object
intensity         object
tmax              object
year               int64
dtype: object

In [28]:
def date_search(df, date):

    event_id_list = []
    event_dates_list = []
    city_id_list = []
    
    for index, row in df.iterrows():
        if date in row['event_dates']:
            
            event_id = row['Event_ID']
            event_dates = row['event_dates']
            city_id = row['ID_HDC_G0']
            
            event_id_list.append(event_id)
            event_dates_list.append(event_dates)
            city_id_list.append(city_id)
    
    df_out = pd.DataFrame()
    df_out['ID_HDC_G0'] = city_id_list
    #df_out['Event_ID'] = event_id_list
    df_out['event_dates'] = event_dates_list
    
    return df_out

In [29]:
df_1231 = date_search(df, '12.31')

In [30]:
df_1231

,ID_HDC_G0,event_dates
0,4449,['1984.12.31']
1,4384,['1984.12.31']
2,4342,['1984.12.31']
3,4300,['1984.12.31']
4,4301,['1984.12.31']
5,4483,['1984.12.31']
6,4227,['1984.12.31']
7,4150,['1984.12.31']
8,4218,['1984.12.31']
9,4320,['1984.12.31']


In [31]:
df_0101 = date_search(df, '01.01')

In [32]:
df_0101.head()

,ID_HDC_G0,event_dates
0,5534,['1984.01.01' '1984.01.02']
1,5541,['1984.01.01' '1984.01.02']
2,5520,['1984.01.01' '1984.01.02']
3,5390,['1984.01.01']
4,3398,['1985.01.01' '1985.01.02']


In [33]:
print(len(df_0101))
print(len(df_1231))

94
97


In [35]:
# Code does not work

# df_overlap1 = df_0101.merge(df_1231, on ='ID_HDC_G0', how = 'inner')
# df_overlap1

## NEXT:

I need to write a function that compares years with the resulting subset of Dec 31 and Jan 1 and then I think we will just drop all these from the record and then add them on as heat waves that overlap years later. There should be about 100 of them